In [ ]:
import sys, os
sys.path.append(os.path.abspath("src"))

In [ ]:
# setup & imports

import os
from pprint import pprint

# Make sure we can import from src/
import sys
sys.path.append(os.path.abspath("src"))
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from src.workflows.meta_workflow import build_meta_graph
from src.workflows.user_workflow import build_user_analytics_graph
from src.workflows.phase2_qna_workflow import build_qna_graph

from pathlib import Path

def clear_all_plots(plot_dir: str = "plots"):
    """Delete all existing PNG files in the plots folder so we always use fresh ones."""
    p = Path(plot_dir)
    if not p.exists():
        return

    removed = 0
    for png in p.glob("*.png"):
        try:
            png.unlink()
            removed += 1
        except Exception as e:
            print(f"Could not delete {png}: {e}")
    print(f"Cleared {removed} old plot(s) from '{plot_dir}'.")

clear_all_plots("plots")


In [ ]:
# run Phase 0 (META) once

meta_graph = build_meta_graph()

print("=== RUNNING META WORKFLOW (PHASE 0) ===")
meta_state = meta_graph.invoke({}, config={"recursion_limit": 80})

print("Meta state keys:", list(meta_state.keys()))
print("Meta summary:")
pprint(meta_state.get("meta_analytics", {}).get("summary", {}))
print("\n meta_llm_tables keys:")
print(list(meta_state.get("meta_llm_tables", {}).keys()))


In [ ]:
# run Phase 1 (USER) once

user_graph = build_user_analytics_graph()

# You can change this to another tag (your)
player_tag = os.getenv("PLAYER_TAG", "8C8JJQLG")

print("=== RUNNING USER WORKFLOW (PHASE 1) ===")
user_state = user_graph.invoke({"player_tag": player_tag})

print("User state keys:", list(user_state.keys()))
print("User summary:")
pprint(user_state.get("user_analytics", {}).get("summary", {}))

print("\n user_llm_tables keys:")
print(list(user_state.get("user_llm_tables", {}).keys()))


In [ ]:
from pathlib import Path
from IPython.display import display, Image

PLOTS_DIR = Path("plots")

def show_named_plots(title: str, filenames: list[str], width: int = 650):
    """Display plots cleanly resized so the notebook looks good."""
    print(f"\n============================")
    print(f"   {title}")
    print(f"============================")

    if not PLOTS_DIR.exists():
        print(f"No '{PLOTS_DIR}' folder found.")
        return

    for fname in filenames:
        full_path = PLOTS_DIR / fname
        print(f"\n{fname}")
        if full_path.exists():
            display(Image(filename=str(full_path), width=width))
        else:
            print(f"[Missing] {full_path}")

# --- Phase 0: Meta plots ---
meta_plot_files = [
    "meta_deck_types_winrate.png",
    "meta_deck_types.png",
    "meta_matchups_bait.png",
    "meta_matchups_beatdown.png",
    "meta_matchups_bridge_spam.png",
    "meta_matchups_cycle.png",
    "meta_matchups_hybrid.png",
    "meta_matchups_siege.png",
]

show_named_plots("Phase 0: Meta plots", meta_plot_files)

# --- Phase 1: User plots ---
user_plot_files = [
    "user_best_cards.png",
    "user_easy_opp_cards.png",
    "user_my_deck_types.png",
    "user_opp_deck_types_bar.png",
    "user_opp_deck_types.png",
    "user_tough_opp_cards.png",
    "user_worst_cards.png",
]

show_named_plots("Phase 1: User plots", user_plot_files)


In [ ]:
#Build Phase 2 Q&A graph

print("=== BUILDING PHASE 2 Q&A GRAPH ===")
qna_graph = build_qna_graph()
print("Q&A graph ready.")


In [ ]:
# Cell 5: interactive Q&A loop (uses cached meta_state & user_state)

def ask_once(question: str):
    """Helper to run a single Q&A turn through the Phase 2 graph."""
    qna_input = {
        "user_tag": user_state.get("player_tag"),
        "question": question,
        "user_analytics": user_state.get("user_analytics", {}),
        "user_llm_tables": user_state.get("user_llm_tables", {}),
        "meta_analytics": meta_state.get("meta_analytics", {}),
        "meta_llm_tables": meta_state.get("meta_llm_tables", {}),
        "meta_table": meta_state.get("meta_table", []),
        "notes": [],
    }

    qna_state = qna_graph.invoke(qna_input)

    # Pull out values to display
    question_text = qna_state.get("question", "")
    category = qna_state.get("question_category", "unknown")
    context_text = qna_state.get("context_text", "").strip()
    answer = qna_state.get("answer", "(no answer)")
    notes = qna_state.get("notes", [])

    # USER-FRIENDLY CLEAN OUTPUT
    print(f"\n[Question]\n{question_text}")
    print(f"\n[Category]\n{category}")

    if context_text:
        print(f"\n[Context text]\n{context_text}")

    print("\n=== ANSWER ===\n")
    print(answer)

    # Keep only short internal trace
    print("\n[Last notes]")
    for line in notes[-5:]:
        print(" •", line)


# Simple REPL-style loop
while True:
    q = input("\nAsk a Clash question (or type 'quit' to exit):\n> ").strip()
    if q.lower() in {"quit", "exit"}:
        print("Goodbye!")
        break

    ask_once(q)


In [ ]:
what deck do I lost to the most?